In [9]:
suppressMessages(library(ggplot2))
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(rtracklayer))
suppressMessages(library(BSgenome.Hsapiens.UCSC.hg38))

In [10]:
set.seed(42)
addArchRThreads(threads = 64)

Setting default number of Parallel threads to 64.



In [11]:
epi_fragments <- readRDS("./ArchRProject_epithelial/epithelial_gr.rds")
blacklist <- import.bed("/data/hanxue/BCY_ATAC/blacklist/hg38-blacklist.v2.bed")

In [12]:
epi_fragments

GRanges object with 103223445 ranges and 2 metadata columns:
                seqnames      ranges strand |                 RG read_count
                   <Rle>   <IRanges>  <Rle> |        <character>  <integer>
          [1]       chr1 10086-10346      * | CGTACAAAGTGTTCCA-1          1
          [2]       chr1 13283-13362      * | GAGTGAGGTGGCATAG-1          3
          [3]       chr1 47091-47138      * | GCTGAGCAGGCCTAAG-1          1
          [4]       chr1 55970-56010      * | GAGGATGTCGGGAAAC-1          2
          [5]       chr1 64822-64979      * | CAAAGCTCAGGCATTT-1          1
          ...        ...         ...    ... .                ...        ...
  [103223441] KI270713.1 34128-34171      * | GACTAGTAGGTGTTGG-1          3
  [103223442] KI270713.1 36867-36905      * | TAAGCCACAACGGGTA-1          6
  [103223443] KI270713.1 36872-36916      * | CAACCAAGTACGAGAC-1          1
  [103223444] KI270713.1 36877-36928      * | GAGCGCTGTTATCGAC-1          1
  [103223445] KI270713.1 37

In [13]:
"%ni%" <- Negate("%in%")

In [14]:
countInsertions <- function(query, fragments, by = "RG"){
  # Count By Fragments Insertions
  # 统计在给定基因组查询区域query（指windows）中的fragments的插入数（插入数：每个fragment片段有start和end两个插入点）

  # inserts（start-end）
  inserts <- c(
    GRanges(seqnames = seqnames(fragments), ranges = IRanges(start(fragments), start(fragments)), RG = mcols(fragments)[,by]),
    GRanges(seqnames = seqnames(fragments), ranges = IRanges(end(fragments), end(fragments)), RG = mcols(fragments)[,by])
  )
  # 重叠计算 - query与inserts之间的重叠
  by <- "RG"
  overlapDF <- DataFrame(findOverlaps(query, inserts, ignore.strand = TRUE, maxgap=-1L, minoverlap=0L, type = "any"))
  overlapDF$name <- mcols(inserts)[overlapDF[, 2], by]
  overlapTDF <- transform(overlapDF, id = match(name, unique(name)))
  # Calculate Overlap Stats
  # 计算重叠统计
  ## inPeaks：在query中的插入位置
  ## total：在query中的总插入数
  ## frip：在query中的inPeaks占总插入数的比例
  inPeaks <- table(overlapDF$name)
  total <- table(mcols(inserts)[, by])
  total <- total[names(inPeaks)]
  frip <- inPeaks / total
  # Summarize
  # 生成稀疏矩阵
  ## 行表示查询区域
  ## 列表示插入位置
  sparseM <- Matrix::sparseMatrix(
    i = overlapTDF[, 1], 
    j = overlapTDF[, 4],
    x = rep(1, nrow(overlapTDF)), 
    dims = c(length(query), length(unique(overlapDF$name))))
  colnames(sparseM) <- unique(overlapDF$name)
  total <- total[colnames(sparseM)]
  frip <- frip[colnames(sparseM)]
  out <- list(counts = sparseM, frip = frip, total = total)
  return(out)
}

In [15]:
makeWindows <- function(genome, blacklist, windowSize = 10e6, slidingSize = 2e6){
	# 在给定基因组（参考基因组）中生成窗口（windowSize, slidingSize），过滤黑名单区域，添加核苷酸信息（GC、AT含量）
	## genome：参考基因组
	## blacklist：黑名单
	## windowSize：窗口大小，默认10Mb
	## slidingSize：滑动步长，默认2Mb
	chromSizes <- GRanges(names(seqlengths(genome)), IRanges(1, seqlengths(genome)))
	### 去除非标准染色体
	chromSizes <- GenomeInfoDb::keepStandardChromosomes(chromSizes, pruning.mode = "coarse")
	windows <- slidingWindows(x = chromSizes, width = windowSize, step = slidingSize) %>% unlist %>% .[which(width(.)==windowSize),]
	mcols(windows)$wSeq <- as.character(seqnames(windows))
  	mcols(windows)$wStart <- start(windows)
  	mcols(windows)$wEnd <- end(windows)
	message("Subtracting Blacklist...")
	### 去掉黑名单区域
	windowsBL <- lapply(seq_along(windows), function(x){
			if(x %% 100 == 0){
				message(sprintf("%s of %s", x, length(windows)))
			}
			gr <- GenomicRanges::setdiff(windows[x,], blacklist)
			mcols(gr) <- mcols(windows[x,])
			return(gr)
		})
	names(windowsBL) <- paste0("w",seq_along(windowsBL))
	windowsBL <- unlist(GRangesList(windowsBL), use.names = TRUE)
	mcols(windowsBL)$name <- names(windowsBL)
	### 添加核苷酸信息
	message("Adding Nucleotide Information...")
	windowSplit <- split(windowsBL, as.character(seqnames(windowsBL)))
	windowNuc <- lapply(seq_along(windowSplit), function(x){
		message(sprintf("%s of %s", x, length(windowSplit)))
	    chrSeq <- Biostrings::getSeq(genome,chromSizes[which(seqnames(chromSizes)==names(windowSplit)[x])])
	    grx <- windowSplit[[x]]
	    aFreq <- alphabetFrequency(Biostrings::Views(chrSeq[[1]], ranges(grx)))
	    mcols(grx)$GC <- rowSums(aFreq[, c("G","C")]) / rowSums(aFreq)
	    mcols(grx)$AT <- rowSums(aFreq[, c("A","T")]) / rowSums(aFreq)
	    return(grx)
	  }) %>% GRangesList %>% unlist %>% sortSeqlevels %>% sort
	### N非GC和AT的比例
	windowNuc$N <- 1 - (windowNuc$GC + windowNuc$AT)
	windowNuc
}

In [16]:
scCNA <- function(windows, fragments, neighbors = 100, LFC = 1.5, FDR = 0.1, force = FALSE, remove = c("chrM","chrX","chrY")){
	# 从scATAC-seq数据中推断CNA
	## windows：一个 GRanges 对象，表示参考基因组生成的窗口。
	## fragments：一个 GRanges 对象，表示 scATAC-seq 数据的片段
	## neighbors：用于背景计算的邻居数（默认值为100）即为GC相似的邻居数
	## LFC：log2FC的阈值（默认值为1.5）
	## FDR：假发现率的阈值（默认值为0.1）
	## force：如果背景均值为0时是否强制继续计算
	## remove：需要去除的染色体（如c("chrM","chrX","chrY")）
	# 只保存标准染色体上的区域
	windows   <- GenomeInfoDb::keepStandardChromosomes(windows, pruning.mode = "coarse")
	fragments <- GenomeInfoDb::keepStandardChromosomes(fragments, pruning.mode = "coarse")
	windows <- windows[seqnames(windows) %ni% remove]
	fragments <- fragments[seqnames(fragments) %ni% remove]

	# 计算窗口内的插入数
	message("Getting Counts...")
	counts <- countInsertions(windows, fragments, by = "RG")[[1]]
	message("Summarizing...")
	# 总结窗口信息：长度、GC、AT、N含量等，保存在windowSummary中
	# windowSummary <- GenomicRangesList()
	windowSummary <- GRangesList()
	countSummary <- matrix(nrow=length(unique(windows$name)), ncol = ncol(counts))
	for(x in seq_along(unique(mcols(windows)$name))){
		if(x %% 100 == 0){
			message(sprintf("%s of %s", x, length(unique(mcols(windows)$name))))
		}
		idx <- which(mcols(windows)$name == unique(mcols(windows)$name)[x])
		wx <- windows[idx,]
		wo <- GRanges(mcols(wx)$wSeq , ranges = IRanges(mcols(wx)$wStart, mcols(wx)$wEnd))[1,]
		mcols(wo)$name <- mcols(wx)$name[1]
		mcols(wo)$effectiveLength <- sum(width(wx))
		mcols(wo)$percentEffectiveLength <- 100*sum(width(wx))/width(wo)
		mcols(wo)$GC <- sum(mcols(wx)$GC * width(wx))/width(wo)
		mcols(wo)$AT <- sum(mcols(wx)$AT * width(wx))/width(wo)
		mcols(wo)$N <- sum(mcols(wx)$N * width(wx))/width(wo)
		countSummary[x,] <- Matrix::colSums(counts[idx,,drop=FALSE])
		windowSummary[[x]] <- wo
	}
	windowSummary <- unlist(windowSummary)
	
	# Keep only regions with less than 0.1% N
	# 过滤低质量窗口，即N含量大于0.1%的窗口
	keep <- which(windowSummary$N < 0.001) 
	windowSummary <- windowSummary[keep,]
	countSummary <- countSummary[keep,]
	
	# Now determine the nearest neighbors by GC content
	# 计算背景
	message("Computing Background...")
	### 初始化，增加背景均值、标准差、log2FC、z值、p值
	bdgMean <- matrix(nrow=nrow(countSummary), ncol=ncol(countSummary))
	bdgSd <- matrix(nrow=nrow(countSummary), ncol=ncol(countSummary))
	log2FC <- matrix(nrow=nrow(countSummary), ncol=ncol(countSummary))
	z <- matrix(nrow=nrow(countSummary), ncol=ncol(countSummary))
	pval <- matrix(nrow=nrow(countSummary), ncol=ncol(countSummary))

	for(x in seq_len(nrow(countSummary))){
		if(x %% 100 == 0){
			message(sprintf("%s of %s", x, nrow(countSummary)))
		}
		## 确定GC含量最接近的windows作为背景。
		#Get Nearest Indices
		idxNN <- head(order(abs(windowSummary$GC[x] - windowSummary$GC)), neighbors + 1)
		idxNN <- idxNN[idxNN %ni% x]
		#Background
		## 计算 
		if(any(colMeans(countSummary[idxNN, ])==0)){
			if(force){
				message("Warning! Background Mean = 0 Try a higher neighbor count or remove cells with 0 in colMins")
			}else{
				stop("Background Mean = 0!")
			}
		}
		bdgMean[x, ] <- colMeans(countSummary[idxNN, ])
		bdgSd[x, ] <- matrixStats::colSds(countSummary[idxNN, ])
		log2FC[x, ] <- log2((countSummary[x, ]+1e-5) / (bdgMean[x, ]+1e-5))
		z[x, ] <- (countSummary[x,] - bdgMean[x, ]) / bdgSd[x, ]
		pval[x, ] <- 2*pnorm(-abs(z[x, ]))
	}
	## 调整p值
	padj <- apply(pval, 2, function(x) p.adjust(x, method = "fdr"))
	## CNA
	CNA <- matrix(0, nrow=nrow(countSummary), ncol=ncol(countSummary))
	### 满足条件的设置为1
	CNA[which(log2FC >= LFC & padj <= FDR)] <- 1
	## 返回SummarizedExperiment对象
	se <- SummarizedExperiment(
		assays = SimpleList(
				CNA = CNA,
				counts = countSummary,
				log2FC = log2FC,
				padj = padj,
				pval = pval,
				z = z,
				bdgMean = bdgMean,
				bdgSd = bdgSd
			),
		rowRanges = windowSummary
	)
	colnames(se) <- colnames(counts)

	return(se)
}

### scATAC-seq推测拷贝数变异
 - 基因组上较大间隔（bins）上的read counts。
 - GC-matched 平均read counts
 

In [17]:
windows <- makeWindows(genome = BSgenome.Hsapiens.UCSC.hg38, blacklist)

Subtracting Blacklist...

100 of 1438

200 of 1438

300 of 1438

400 of 1438

500 of 1438

600 of 1438

700 of 1438

800 of 1438

900 of 1438

1000 of 1438

1100 of 1438

1200 of 1438

1300 of 1438

1400 of 1438

Adding Nucleotide Information...

1 of 24

2 of 24

3 of 24

4 of 24

5 of 24

6 of 24

7 of 24

8 of 24

9 of 24

10 of 24

11 of 24

12 of 24

13 of 24

14 of 24

15 of 24

16 of 24

17 of 24

18 of 24

19 of 24

20 of 24

21 of 24

22 of 24

23 of 24

24 of 24



In [18]:
cnaObj <- scCNA(windows, epi_fragments, neighbors = 100, LFC = 1.5, FDR = 0.1, force = FALSE, remove = c("chrM","chrX","chrY"))

Getting Counts...

Summarizing...

100 of 1323

Warning message in .merge_two_Seqinfo_objects(x, y):
“The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)”
200 of 1323

Warning message in .merge_two_Seqinfo_objects(x, y):
“The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)”
300 of 1323

Warning message in .merge_two_Seqinfo_objects(x, y):
“The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)”
400 of 1323

Warning message in .merge_two_Seqinfo_objects(x, y):
“The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)”
500 of 1323

Warning message in .merge_two_Seqinfo_objects(x, y):
“The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)”
Warning message in .merge_two_Seqinfo_objects(x, y):
“The 2 combined o

In [19]:
saveRDS(cnaObj, "./ArchRProject_epithelial/epithelial_cna.rds")